In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
import shap

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import os

np.random.seed(42)
EPS = 1e-9

## Dataset

本プロジェクトで使用しているデータセットは以下の公開データです：

 **IBM HR Analytics Attrition Dataset**  
https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset

Kaggle でアカウントログイン後にダウンロードし、
ノートブックと同じフォルダに配置して実行してください。

例：  
```bash
cp ~/Downloads/WA_Fn-UseC_-HR-Employee-Attrition.csv ./data/

In [ ]:
#個人のディレクトリ構造に合わせてパスを変更
#path = '.../WA_Fn-UseC_-HR-Employee-Attrition.csv'
df = pd.read_csv(path)

##前処理


In [ ]:
print(df.columns.tolist())

In [ ]:
df.isna().sum()[df.isna().sum()>0]

In [ ]:
df.dtypes

In [ ]:
#目的変数
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

#前処理
df['OverTime'] = df['OverTime'].map({'Yes': 1, 'No': 0})

df['BusinessTravel'] = df['BusinessTravel'].map({
    'Non-Travel': 0,
    'Travel_Rarely': 1,
    'Travel_Frequently': 2
})

travel_score = {
    "Non-Travel": 0,
    "Travel_Rarely": 1,
    "Travel_Frequently": 2,
}

##特徴量作成

In [ ]:
## Age 設計

df["is_young"] = (df["Age"] < 30).astype(int)
df["is_mid"]   = ((df["Age"] >= 30) & (df["Age"] < 45)).astype(int)
df["is_senior"]= (df["Age"] >= 45).astype(int)

# キャリア年数との相対
df["career_density"] = df["TotalWorkingYears"] / (df["Age"] + EPS)

# 年齢×役職
df["age_x_joblevel"] = df["Age"] * df["JobLevel"]


## JobRole 設計

jobrole_map = {
    #  技術/専門寄り
    "Research Scientist": "technical",
    "Laboratory Technician": "technical",
    "Manufacturing Director": "technical",
    "Healthcare Representative": "technical",

    #  営業/対外寄り
    "Sales Executive": "customer_facing",
    "Sales Representative": "customer_facing",
    "Human Resources": "internal_support",

    #  管理/統括寄り
    "Manager": "managerial",
}

df["JobRole_group"] = df["JobRole"].map(jobrole_map).fillna("other")

# 管理職かどうか
df["is_manager_role"] = (df["JobRole"] == "Manager").astype(int)

## 報酬系の統合・相対化

# 月収×役職
df["income_per_level"] = df["MonthlyIncome"] / (df["JobLevel"] + 1)

# 昇給率と収入の関係
df["hike_x_income"] = df["PercentSalaryHike"] * np.log1p(df["MonthlyIncome"])

# 「日給/時給」など生値が多い場合は、代表だけ残す or 比率へ
df["daily_to_monthly"] = df["DailyRate"] / (df["MonthlyIncome"] + 1)


## 努力–報酬ギャップ

# 残業しているのに昇給が低い
df["overtime_reward_gap"] = df["OverTime"] / (df["PercentSalaryHike"] + EPS)

# 役職の割に収入が低い
df["level_income_gap"] = df["JobLevel"] / (np.log1p(df["MonthlyIncome"]) + EPS)

# 年収水準に対する昇給
df["hike_per_income"] = df["PercentSalaryHike"] / (np.log1p(df["MonthlyIncome"]) + EPS)

## ストレス構造
df['stress_proxy'] = (
    (4 - df['WorkLifeBalance'])
    + (4 - df['EnvironmentSatisfaction'])
    + (4 - df['RelationshipSatisfaction'])
)

df['high_stress_overtime'] = df['stress_proxy'] * df['OverTime']


## 働き方ギャップ

df["BusinessTravel_score"] = df["BusinessTravel"].map(travel_score).fillna(0).astype(int)

# 出張×距離（移動負荷）
df["travel_distance_gap"] = df["BusinessTravel"] * df["DistanceFromHome"]

# 残業×距離（生活摩擦）
df["overtime_distance_gap"] = df["OverTime"] * df["DistanceFromHome"]

# マネジメント接点
df['manager_exposure_ratio'] = (
    df['YearsWithCurrManager'] / (df['YearsAtCompany'] + 1)
)

df['young_low_support_flag'] = (
    (df['Age'] < 30).astype(int)
    * (df['YearsWithCurrManager'] <= 1).astype(int)
)

# 停滞（昇進無し）×上司同じ
df["stagnation_manager_lock"] = df["YearsSinceLastPromotion"] * df["YearsWithCurrManager"]


特徴量設計の思想と各変数の設計意図

本分析では、離職を 年齢や勤続年数といった単純属性の問題として捉えるのではなく、
報酬・業務負荷・マネジメント接点といった組織構造上のギャップとして捉えることを目的に
特徴量設計を行った。

1. Age 系特徴量（年齢そのものを使わないための設計）
設計方針

年齢は情報量が大きい一方で、
直接使うと 属性依存モデルになりやすい。
施策に落としにくい。
倫理的・実務的にも扱いづらい
という問題がある。
そのため本分析では、
年齢そのものではなく「年齢が組織構造の中でどう位置づけられているか」を
他の変数との関係で表現することを意図した。

is_young / is_mid / is_senior

設計意図
ライフステージの違いによる傾向差を、
連続値ではなく 粗いカテゴリとして限定的に取り込むため。
若年層：キャリア初期の不安定性
中堅層：停滞・期待とのギャップ
高年層：役割固定化
を区別するための補助的指標。

career_density
（TotalWorkingYears / Age）
設計意図
年齢ではなく、「年齢に対して、どれだけキャリアが詰まっているか」を表現するため。
値が高い：若い割に経験が多い（昇進期待が高まりやすい）
値が低い：年齢に対して経験が少ない（停滞感・不安）

age_x_joblevel
設計意図
同じ年齢でも、その年齢でその役職にいることが期待通りか／遅れているかは異なるため。
年齢 × 役職という相互作用により「年齢だけ」「役職だけ」では捉えられない 階層的不整合を表現する。


2. JobRole 設計（役割の意味を集約する）
設計方針

JobRole はカテゴリ数が多く、そのまま使うと疎。
解釈がしづらいため、役割の性質ベースで集約した。

JobRole_group
設計意図
職種名ではなく、技術・専門職、顧客対応職、管理・統括職、内部支援といった 業務特性の違いを捉えるため。
これにより、離職が「職種名」ではなく「業務の性質」に起因しているかを分析可能にした。

is_manager_role
設計意図
管理職は、責任、評価基準、ストレス構造が非管理職と大きく異なるため、明示的なフラグとして切り出すことを目的とした。


3. 報酬系の統合・相対化
学術的背景

努力–報酬不均衡モデル
（Effort–Reward Imbalance Model：Siegrist, 1996）
人は「努力量」そのものではなく、努力と報酬の釣り合いに強く反応する

income_per_level
設計意図
絶対収入ではなく、「その役職に対して、どれくらいの収入か」という 公平感・納得感を表現するため。

hike_x_income
設計意図
昇給率は単体では弱いため、昇給率 × 収入水準とすることで、「昇給が実質的に効いているか」を表現。

daily_to_monthly
設計意図
日給・月給など複数の報酬指標がある中で、生値を並べるのではなく相対比率として整理するための冗長性削減。


4. 努力–報酬ギャップ
overtime_reward_gap
設計意図
残業しているにもかかわらず、
昇給が低い状態を 不公平感の代理指標として表現。

level_income_gap
設計意図
役職に対して収入が見合っていないと感じる状況を数値化するため。

hike_per_income
設計意図
昇給率が高くても、
収入水準が高いと 実感が薄れることを考慮し、「昇給がどれくらい効いているか」を相対化。


5. ストレス構造
学術的背景
職務ストレス理論（Job Demand–Resources Model など）
ストレスは単一要因ではなく複数要因の累積で生じる。

stress_proxy
設計意図
職場環境・人間関係・WLB を単一の心理的負荷指標として統合。

high_stress_overtime
設計意図
ストレスと残業は 同時に発生したときに影響が増幅するため、単純な加算ではなく 相互作用として表現。

6. 働き方ギャップ
travel_distance_gap
設計意図
出張頻度と通勤距離を掛け合わせることで、「移動に伴う生活摩擦」を定量化。

overtime_distance_gap
設計意図
残業 × 通勤距離により、「仕事が生活時間を侵食している度合い」を表現。


7. マネジメント接点
manager_exposure_ratio
設計意図
在籍期間の中で、「どれだけ同じ上司と接してきたか」を割合として表現。
低い：支援不足・関係未構築
高い：関係固定化・閉塞
の両面を捉える。

young_low_support_flag
設計意図
若手 × 上司経験が浅い
という 支援リスクが高い状態を明示的に表現。

stagnation_manager_lock
設計意図
昇進がなく、かつ上司が変わらない状態をキャリア停滞の代理指標として定義。

In [ ]:
# 不要な列の除外

#SHAP用に保存
employee_ids = df['EmployeeNumber']

drop_cols = [
    'EmployeeNumber',
    'EmployeeCount',
    'Over18',
    'StandardHours',
    'Department',
    'MaritalStatus',
    'OverTime',
    "MonthlyRate",
    "DailyRate",
    "HourlyRate",
    "daily_to_monthly",
    "JobRole",
    "Age",
    "YearsAtCompany",
    "TotalWorkingYears",
    "DistanceFromHome",
    "MonthlyIncome",
    "PercentSalaryHike",
    "NumCompaniesWorked",

]


df = df.drop(columns=drop_cols, errors='ignore')


# カテゴリ変数のエンコード

cat_cols = df.select_dtypes(include='object').columns

for col in cat_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

特徴量補足：成果物として“説明変数に入れる正当性”を考えて設計
方針:まずは残して比較（AUC）→ 変わらなければ落とす
(もしくは「説明では使わない」運用)

EmployeeNumber:削除、ただし個人SHAP分析用にemployee_idsとして別管理

EmployeeCount / Over18 / StandardHours:定数・情報なし

MaritalStatus / Gender:予測には効く場合があるが、施策説明に使いにくい／センシティブ

Department:AUCに影響が小さい。JobRoleと同様に「ラベル暗記」になりがち。

OverTime/DistanceFromHome/MonthlyIncome/income_per_level/hike_x_income:SHAPで類似特徴量と相互作用が強く出ているため、単体は冗長になりやすい。


In [ ]:
# 学習データ作成

X = df.drop('Attrition', axis=1)
y = df['Attrition']

X_train, X_test, y_train, y_test,id_train, id_test = train_test_split(
    X, y, employee_ids, #データに社員番号を紐づけ
    test_size=0.2,
    random_state=42,
    stratify=y
)

##RandomizedSearch

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    # 木の複雑さを制御
    "max_depth": [2, 3, 4, 5],
    "num_leaves": [27, 31, 45],

    # 過学習を抑える最重要パラメータ
    "min_child_samples": [100, 110, 120, 150, 170, 200],

    # 学習の安定化（小LR × 多木）
    "learning_rate": [0.07, 0.09, 0.1, 0.15, 0.17, 0.2],
    "n_estimators": [600, 800, 1200],

    # ランダム性で汎化向上
    "subsample": [0.5, 0.7, 0.9, 1.0],
    "subsample_freq": [0, 0.05, 0.1, 0.2, 0.5, 0.8, 1],
    "colsample_bytree": [0.06, 0.08, 0.1, 0.3, 0.5, 0.6, 0.7, 0.9],

    # 正則化
    "reg_alpha":[0.01, 0.05, 0.3, 0.5, 0.7, 0.9],
    "reg_lambda": [0, 1.0, 3.0, 5.0, 7.0, 10.0],

    # 分割の厳しさ
    "min_split_gain": [0.01, 0.002, 0.003, 0.004, 0.05],
}

search = RandomizedSearchCV(
    lgb.LGBMClassifier(
        class_weight="balanced",
        random_state=42
    ),
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=5,
    random_state=42,
    n_jobs=-1
)

search.fit(X_train, y_train)

model = search.best_estimator_

In [ ]:
tuned_params = search.best_params_
print("tuned Parameters:")
for k, v in tuned_params.items():
    print(f"  {k}: {v}")

##LightGBM

In [ ]:
# LightGBM 学習

model = lgb.LGBMClassifier(
    #**tuned_params,

    # final params
    # Performance:
    #   Train ROC-AUC:0.945
    #   Test  ROC-AUC:0.831
    subsample_freq=1,
    subsample=0.9,
    reg_alpha=0.5,
    reg_lambda=5.0,
    num_leaves=30,
    n_estimators=1200,
    min_split_gain=0.01,
    min_child_samples=120,
    max_depth=4,
    learning_rate=0.15,
    colsample_bytree=0.1,
    class_weight='balanced',
    random_state=42

)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric="auc",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=50)
    ]
)

##モデル評価/AUC

In [ ]:
# モデル評価
train_auc = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
test_auc  = roc_auc_score(y_test,  model.predict_proba(X_test)[:, 1])

print(f"Train ROC-AUC: {train_auc:.3f}")
print(f"Test  ROC-AUC: {test_auc:.3f}")

# 特徴量重要度

importances = pd.Series(
    model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

print('\n=== Feature Importance (LightGBM) ===')
display(importances.head(20))

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

y_score = model.predict_proba(X_test)[:, 1]

fpr, tpr, _ = roc_curve(y_test, y_score)
auc_score = roc_auc_score(y_test, y_score)

plt.figure(figsize=(6, 4))
plt.plot(
    fpr, tpr,
    label=f"LightGBM (AUC = {auc_score:.3f})"
)
plt.plot([0, 1], [0, 1], linestyle="--", label="Random")

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (LightGBM)")
plt.legend()
plt.tight_layout()
plt.show()



##SHAP

In [ ]:
# SHAP による寄与度分析

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_train)

# 全体重要度
shap.summary_plot(
    shap_values,
    X_train,
    plot_type='bar',
    max_display=20
)

In [ ]:
# 9-2. 分布付きSHAP
shap.summary_plot(
    shap_values,
    X_train,
    max_display=20
)

##任意の社員に対する SHAP waterfall

In [ ]:
test_scores = model.predict_proba(X_test)[:, 1]

test_summary = pd.DataFrame({
    "EmployeeNumber": id_test.values,
    "AttritionRisk": test_scores
}).sort_values("AttritionRisk", ascending=False)

display(test_summary.head(20))   # 上位20人を表示


##個別分析

In [ ]:
# 社員番号を入力して分析

TARGET_EMPLOYEE_ID = int(input("分析したい社員番号を入力してください: "))

if TARGET_EMPLOYEE_ID in id_test.values:
    target_idx = id_test[id_test == TARGET_EMPLOYEE_ID].index[0]
    x_one = X_test.loc[[target_idx]]
elif TARGET_EMPLOYEE_ID in id_train.values:
    target_idx = id_train[id_train == TARGET_EMPLOYEE_ID].index[0]
    x_one = X_train.loc[[target_idx]]
else:
    raise ValueError("入力した社員番号は train / test のどちらにも存在しません")

print(f"Selected Employee ID: {TARGET_EMPLOYEE_ID}")


In [ ]:
# 個人1名の説明（waterfall）
# 任意の社員番号を指定

# 社員番号から対応する index を取得
target_idx = id_test[id_test == TARGET_EMPLOYEE_ID].index[0]
x_one = X_test.loc[[target_idx]]

# 離職確率の算出（平均 vs 個人）

# 全体（train）における平均的な離職確率
mean_prob = model.predict_proba(X_train)[:, 1].mean()

# 対象社員の離職確率
emp_prob = model.predict_proba(x_one)[0, 1]
print("=== 離職リスク比較 ===")
print(f"平均的な社員の離職確率 : {mean_prob*100:.1f}%")
print(f"対象社員の離職確率     : {emp_prob*100:.1f}%")
print(f"平均との差               : {(emp_prob - mean_prob)*100:+.1f}%")


# その人のSHAPを描画
x_one = X_test.loc[[target_idx]]
shap_one = explainer.shap_values(x_one)
shap_one_pos = shap_one[1] if isinstance(shap_one, list) else shap_one

OUTDIR = "./outputs"
DPI = 300
os.makedirs(OUTDIR, exist_ok=True)

fig = plt.figure()
shap.plots._waterfall.waterfall_legacy(
    explainer.expected_value[1] if isinstance(explainer.expected_value, (list, np.ndarray))
    else explainer.expected_value,
    shap_one_pos[0],
    x_one.iloc[0],
    max_display=15,
    show=False
)
plt.tight_layout()
plt.savefig(
    os.path.join(OUTDIR, f"shap_waterfall_emp_{TARGET_EMPLOYEE_ID}.png"),
    dpi=DPI,
    bbox_inches="tight"
)
plt.show()
plt.close(fig)

print(f"\nSaved figures are in: {OUTDIR}/")
print("Done.")

##一言評価

In [ ]:
shap_df = pd.DataFrame({
    "feature": X.columns,
    "shap": shap_one[0] if shap_one.ndim == 2 else shap_one
})

# 絶対値で寄与度順に並べ替え
shap_df = shap_df.reindex(
    shap_df["shap"].abs().sort_values(ascending=False).index
)

# SHAPコメント生成（簡素版）

def generate_action_comment(risk_df):
    if risk_df is None or len(risk_df) == 0:
        return "特定のリスク要因は検出されなかった。"

    features = " ".join(risk_df["feature"]).lower()

    if any(k in features for k in ["income", "hike", "level_income"]):
        return "報酬と業務負荷のバランスに関する見直しが有効と考えられる。"
    if any(k in features for k in ["manager", "joblevel", "yearswithcurrmanager", "stagnation"]):
        return "マネジメント接点や役割設計の改善が重要である可能性が示唆される。"
    if any(k in features for k in ["overtime", "travel"]):
        return "業務負荷や働き方に関する調整が離職リスク低減に寄与する可能性がある。"

    return "複数要因が重なって離職リスクが形成されていると考えられる。"


def summarize_factors(df, prefix):
    if df is None or len(df) == 0:
        return f"{prefix}要因は限定的である。"

    features = "、".join(df["feature"].tolist())
    return f"{prefix}要因として、{features} が挙げられる。"



# 上位SHAP要因抽出
# ※ shap_df は abs_shap で降順ソート済み前提


TOP_K = 3

risk_factors = shap_df.query("shap > 0").head(TOP_K)
protective_factors = shap_df.query("shap < 0").head(TOP_K)


# コメント生成

risk_comment = summarize_factors(risk_factors, "離職リスクを高める")
protect_comment = summarize_factors(protective_factors, "離職リスクを抑制する")
action_comment = generate_action_comment(risk_factors)

full_comment = f"""【社員ID: {TARGET_EMPLOYEE_ID}】
{risk_comment}
{protect_comment}
{action_comment}
"""

print(full_comment)